In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import simplekml
import requests
import ipaddress
from scapy.all import rdpcap

In [ ]:
# Load the pcap file
packets = rdpcap('wire.pcap')
print('Packet loaded from .pcap file')

In [ ]:
# Extracting the required data from packets
data = []
for packet in packets:
    if packet.haslayer('IP'):
        src_ip= packet['IP'].src
        dst_ip = packet['IP'].dst
        protocol = packet['IP'].proto
        length = packet['IP'].len
        data.append([src_ip, dst_ip, protocol, length])
print('Data had been Extracted')

In [ ]:
# Convert to a DataFrame
df = pd.DataFrame(data, columns=['Source IP', 'Destination IP', 'Protocol', 'Length'])

# Save to CSV
df.to_csv('network_data.csv', index=False)

print('.csv file has been generated')

In [ ]:
# Load the network data
df = pd.read_csv('network_data.csv')

print('.csv has been loaded')

In [ ]:
# Example visualization: Number of packets per protocol
protocol_counts = df['Protocol'].value_counts()

plt.figure(figsize=(10, 6))
protocol_counts.plot(kind='bar')
plt.xlabel('Protocol')
plt.ylabel('Number of Packets')
plt.title('Number of Packets per Protocol')
plt.show()

In [ ]:
# Create a KML object
kml = simplekml.Kml()

print('Kml file object has been initialzed')

#your ipinfo.io API token
api_token = 'Your_API_Token'

In [ ]:
# Dictionary to store coordinates for each IP
ip_coords = {}

print('IP dict has been initialzed')

In [11]:
def get_coords_from_ip(ip, token):
    # Check if the IP is in the specified range
    if ipaddress.ip_address(ip) in ipaddress.ip_network('192.168.0.0/16'):
        return (79.481774,29.218127)
    if ipaddress.ip_address(ip) in ipaddress.ip_network('224.0.0.0/8'):
        return (-0.119386,51.522961)
    # Use ipinfo.io API for other IPs
    url = f"https://ipinfo.io/{ip}/json?token={token}"
    response = requests.get(url)
    data = response.json()
    if 'loc' in data:
        lat, lon = map(float, data['loc'].split(','))
        return (lon, lat)  # KML uses (longitude, latitude)
    else:
        print(f"No coordinates found for IP: {ip}")
        return None

In [12]:
# Add lines between source and destination IPs
for index, row in df.iterrows():
    src_ip = row['Source IP']
    dst_ip = row['Destination IP']

    # Get coordinates for source IP
    if src_ip not in ip_coords:
        src_coords = get_coords_from_ip(src_ip, api_token)
        if src_coords:
            ip_coords[src_ip] = src_coords
        else:
            continue  # Skip if no coordinates found
    src_coords = ip_coords[src_ip]

    # Get coordinates for destination IP
    if dst_ip not in ip_coords:
        dst_coords = get_coords_from_ip(dst_ip, api_token)
        if dst_coords:
            ip_coords[dst_ip] = dst_coords
        else:
            continue  # Skip if no coordinates found
    dst_coords = ip_coords[dst_ip]

    # Add line to KML
    line = kml.newlinestring(name=f"Packet {index+1}",
                             description=f"Source: {src_ip}\nDestination: {dst_ip}\nProtocol: {row['Protocol']}\nLength: {row['Length']}",
                             coords=[src_coords, dst_coords])
    line.style.linestyle.color = simplekml.Color.red
    line.style.linestyle.width = 2

In [ ]:
# Save to KML file
kml.save('network_lines.kml')

print('kml has been genearted')